# Approximation of the cost of the centralized and decentralized systems for a finite N in the SIR model with vaccinations

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import math
import scipy.io as sio

from mpl_toolkits.mplot3d import Axes3D
#import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
#import numpy

The function below computes the mean-field equilibrium

In [3]:
def mf_equilibrium():
    """
    We compute the mean-field equilibrium as a fixed-point problem. The algorithm stops if 
    after n_loops_max the fixed point has not been found.
    """
    count2=0; n_loops_max=5000
    thr_pl0=-1;
    best_pol=np.zeros(TimeHorizon+1);
    vs = np.zeros(TimeHorizon+1); vi = np.zeros(TimeHorizon+1)
    
    def best_response_threshold(thr_mass):
        x_S = np.zeros(TimeHorizon+1); x_I = np.zeros(TimeHorizon+1);
        policy = np.zeros(TimeHorizon+1);
        x_S[0]=S0; x_I[0]=I0;
        for i in range(TimeHorizon):
            if i<thr_mass:
                policy[i]=theta;
            else:
                policy[i]=vac_min;
            x_S[i+1] = x_S[i]+(-gamma*x_I[i]*x_S[i] - policy[i]*x_S[i])/C;
            x_I[i+1] = x_I[i]+(gamma*x_S[i] - rho)*x_I[i]/C;

        for t_ in range(TimeHorizon,0,-1):
            if c_V - vs[t_] < 0:
                best_pol[t_-1]=theta;
                p_s_s=1 - 1.*gamma*x_I[t_-1]/C - 1.*theta/C;
                p_s_i=1.*gamma*x_I[t_-1]/C; 
                vs[t_-1]= theta * c_V /C + p_s_s * vs[t_] + p_s_i * vi[t_];
            else:
                best_pol[t_-1]=vac_min;
                p_s_s=1 - 1.*gamma*x_I[t_-1]/C - vac_min/C;
                p_s_i=gamma*x_I[t_-1]/C; 
                vs[t_-1]= vac_min * c_V/C + p_s_s * vs[t_] + p_s_i * vi[t_];   
            p_i_i=1-rho/C; 
            vi[t_-1] = c_I/C + p_i_i * vi[t_];  
        
        thr=0;
        for i in range(TimeHorizon+1):
            if vs[i]<c_V:
                thr_pl0=i;
                break;
        return thr_pl0
    
    # Now we first find a threshold such that the BR is smaller than the threshold
    thr_mass=0
    while best_response_threshold(thr_mass) > thr_mass:
        thr_mass = max(1,thr_mass*2)
        
    # Then we apply a dicotomic search 
    thr_mass = int(thr_mass/2)
    deltaThreshold=int(np.ceil(thr_mass/2))
    for i in range(20):
        br_threshold = best_response_threshold(thr_mass)
        if br_threshold > thr_mass:
            thr_mass += deltaThreshold
        elif br_threshold < thr_mass:
            thr_mass -= deltaThreshold
        deltaThreshold = min(thr_mass,int(np.ceil(deltaThreshold/2)))
        if deltaThreshold <= 1:
            break
        
    cost_mfe=vs[0]*S0+vi[0]*I0;
    print('threshold=',thr_mass,end=' ')
    return cost_mfe;

#for I0 in [1e-3,1e-9,1e-12,1e-25,1e-28]:
#    mf_equilibrium()

The function above computes the mean-field optimum

In [4]:
def mf_optimum():
    """
    This method uses a divide and conquer approach to compute the minimum value. 
    It assumes that the optimal policy is a bang-bang policy with a unique jump time"""
    my_policy=np.zeros(TimeHorizon+1);
    j=0;
    v=np.zeros(TimeHorizon+1);
    x_S=np.zeros(TimeHorizon+1);
    x_I=np.zeros(TimeHorizon+1);
    
    def compute_cost(t_critical):
        x_S[0]=S0; x_I[0]=I0;
        if v[t_critical] > 0: return
        for i in range(TimeHorizon):
            my_policy[i] = theta if i<t_critical else vac_min
            x_S[i+1] = x_S[i]+(-gamma*x_I[i]*x_S[i] - my_policy[i]*x_S[i])/C;
            x_I[i+1] = x_I[i]+(gamma*x_I[i]*x_S[i] - rho*x_I[i])/C;    

        v[t_critical] = sum(1.*c_I*x_I/C+1.*c_V*x_S*my_policy/C);    
    
    t_critical = int(C/2)
    deltaT = int(C/4)
    for i in range(20):
        for t in [t_critical,t_critical+1]:
            compute_cost(t)
        if v[t_critical+1]>v[t_critical]: 
            t_critical -= deltaT
        else: 
            t_critical += deltaT
        deltaT = min(t_critical,int(np.ceil(deltaT/2)))
        if deltaT == 0:
            break
#    plt.plot(x_S)  
#    plt.semilogy(x_I)
#    #plt.plot(x_I)
#    plt.plot([0,10000],[rho/gamma,rho/gamma],'--')
#    plt.legend(['m_S','m_I'])

        
    cost_opt=v[t_critical]
    mypos=t_critical
    return cost_opt; 

#for I0 in [1e-3,1e-4,1e-5]:
#    mf_optimum()

In [ ]:
# system parameters
gamma = 73.0      # infection rate
rho = 36.5        # recovery rate
vac_min = 0.0     # minimum vaccination rate
theta = 10.0      # maximum vaccination rate
T = 1             # time horizon

# costs
c_V = 0.5       # cost per unit time of vaccination
c_I = 36.5         # cost per unit time of infection

# uniformization constant
C=int(100000)
TimeHorizon = int(T*C)

N=100
mfe_cost = []; mfopt_cost = []
for s0 in range(1,N):
    mfe_cost_v = []; mfopt_cost_v = []
    for i0 in range(1,N):
        print('computing for (s,i)=',s0,i0,end='...')
        if N-s0-i0>0: 
            I0=i0/N; S0=s0/N
            val_mfe_cost = mf_equilibrium()
            print('mfe done',end='...')
            val_mfopt_cost = mf_optimum()
            print('mfopt done')
        else:
            val_mfe_cost = 0
            val_mfopt_cost = -1
        mfe_cost_v.append(val_mfe_cost)
        mfopt_cost_v.append(val_mfopt_cost)
    mfe_cost.append(mfe_cost_v); 
    mfopt_cost.append(mfopt_cost_v)
    
mat_mfe_cost=np.array([np.array(xi) for xi in mfe_cost])
mat_mfopt_cost=np.array([np.array(xi) for xi in mfopt_cost])
sio.savemat('data/mfe_cost_N100.mat', {'mfe_cost':mfe_cost})
sio.savemat('data/mfopt_cost_N100.mat', {'mfopt_cost':mfopt_cost})

computing for (s,i)= 1 1...equi done
computing for (s,i)= 1 2...equi done
computing for (s,i)= 1 3...equi done
computing for (s,i)= 1 4...equi done
computing for (s,i)= 1 5...equi done
computing for (s,i)= 1 6...equi done
computing for (s,i)= 1 7...equi done
computing for (s,i)= 1 8...equi done
computing for (s,i)= 1 9...equi done
computing for (s,i)= 1 10...equi done
computing for (s,i)= 1 11...equi done
computing for (s,i)= 1 12...equi done
computing for (s,i)= 1 13...equi done
computing for (s,i)= 1 14...equi done
computing for (s,i)= 1 15...equi done
computing for (s,i)= 1 16...equi done
computing for (s,i)= 1 17...equi done
computing for (s,i)= 1 18...equi done
computing for (s,i)= 1 19...equi done
computing for (s,i)= 1 20...equi done
computing for (s,i)= 1 21...equi done
computing for (s,i)= 1 22...equi done
computing for (s,i)= 1 23...equi done
computing for (s,i)= 1 24...equi done
computing for (s,i)= 1 25...equi done
computing for (s,i)= 1 26...equi done
computing for (s,i)= 

computing for (s,i)= 3 20...equi done
computing for (s,i)= 3 21...equi done
computing for (s,i)= 3 22...equi done
computing for (s,i)= 3 23...equi done
computing for (s,i)= 3 24...equi done
computing for (s,i)= 3 25...equi done
computing for (s,i)= 3 26...equi done
computing for (s,i)= 3 27...equi done
computing for (s,i)= 3 28...equi done
computing for (s,i)= 3 29...equi done
computing for (s,i)= 3 30...equi done
computing for (s,i)= 3 31...equi done
computing for (s,i)= 3 32...equi done
computing for (s,i)= 3 33...equi done
computing for (s,i)= 3 34...equi done
computing for (s,i)= 3 35...equi done
computing for (s,i)= 3 36...equi done
computing for (s,i)= 3 37...equi done
computing for (s,i)= 3 38...equi done
computing for (s,i)= 3 39...equi done
computing for (s,i)= 3 40...equi done
computing for (s,i)= 3 41...equi done
computing for (s,i)= 3 42...equi done
computing for (s,i)= 3 43...equi done
computing for (s,i)= 3 44...equi done
computing for (s,i)= 3 45...equi done
computing fo

computing for (s,i)= 5 40...equi done
computing for (s,i)= 5 41...equi done
computing for (s,i)= 5 42...equi done
computing for (s,i)= 5 43...equi done
computing for (s,i)= 5 44...equi done
computing for (s,i)= 5 45...equi done
computing for (s,i)= 5 46...equi done
computing for (s,i)= 5 47...equi done
computing for (s,i)= 5 48...equi done
computing for (s,i)= 5 49...equi done
computing for (s,i)= 5 50...equi done
computing for (s,i)= 5 51...equi done
computing for (s,i)= 5 52...equi done
computing for (s,i)= 5 53...equi done
computing for (s,i)= 5 54...equi done
computing for (s,i)= 5 55...equi done
computing for (s,i)= 5 56...equi done
computing for (s,i)= 5 57...equi done
computing for (s,i)= 5 58...equi done
computing for (s,i)= 5 59...equi done
computing for (s,i)= 5 60...equi done
computing for (s,i)= 5 61...equi done
computing for (s,i)= 5 62...equi done
computing for (s,i)= 5 63...equi done
computing for (s,i)= 5 64...equi done
computing for (s,i)= 5 65...equi done
computing fo

computing for (s,i)= 7 61...equi done
computing for (s,i)= 7 62...equi done
computing for (s,i)= 7 63...equi done
computing for (s,i)= 7 64...equi done
computing for (s,i)= 7 65...equi done
computing for (s,i)= 7 66...equi done
computing for (s,i)= 7 67...equi done
computing for (s,i)= 7 68...equi done
computing for (s,i)= 7 69...equi done
computing for (s,i)= 7 70...equi done
computing for (s,i)= 7 71...equi done
computing for (s,i)= 7 72...equi done
computing for (s,i)= 7 73...equi done
computing for (s,i)= 7 74...equi done
computing for (s,i)= 7 75...equi done
computing for (s,i)= 7 76...equi done
computing for (s,i)= 7 77...equi done
computing for (s,i)= 7 78...equi done
computing for (s,i)= 7 79...equi done
computing for (s,i)= 7 80...equi done
computing for (s,i)= 7 81...equi done
computing for (s,i)= 7 82...equi done
computing for (s,i)= 7 83...equi done
computing for (s,i)= 7 84...equi done
computing for (s,i)= 7 85...equi done
computing for (s,i)= 7 86...equi done
computing fo

computing for (s,i)= 9 83...equi done
computing for (s,i)= 9 84...equi done
computing for (s,i)= 9 85...equi done
computing for (s,i)= 9 86...equi done
computing for (s,i)= 9 87...equi done
computing for (s,i)= 9 88...equi done
computing for (s,i)= 9 89...equi done
computing for (s,i)= 9 90...equi done
computing for (s,i)= 9 91...computing for (s,i)= 9 92...computing for (s,i)= 9 93...computing for (s,i)= 9 94...computing for (s,i)= 9 95...computing for (s,i)= 9 96...computing for (s,i)= 9 97...computing for (s,i)= 9 98...computing for (s,i)= 9 99...computing for (s,i)= 10 1...equi done
computing for (s,i)= 10 2...equi done
computing for (s,i)= 10 3...equi done
computing for (s,i)= 10 4...equi done
computing for (s,i)= 10 5...equi done
computing for (s,i)= 10 6...equi done
computing for (s,i)= 10 7...equi done
computing for (s,i)= 10 8...equi done
computing for (s,i)= 10 9...equi done
computing for (s,i)= 10 10...equi done
computing for (s,i)= 10 11...equi done
computing for (s,i)= 10 

...

computing for (s,i)= 14 24...equi done
computing for (s,i)= 14 25...equi done
computing for (s,i)= 14 26...equi done
computing for (s,i)= 14 27...equi done
computing for (s,i)= 14 28...equi done
computing for (s,i)= 14 29...equi done
computing for (s,i)= 14 30...equi done
computing for (s,i)= 14 31...equi done
computing for (s,i)= 14 32...equi done
computing for (s,i)= 14 33...equi done
computing for (s,i)= 14 34...equi done
computing for (s,i)= 14 35...equi done
computing for (s,i)= 14 36...equi done
computing for (s,i)= 14 37...equi done
computing for (s,i)= 14 38...equi done
computing for (s,i)= 14 39...equi done
computing for (s,i)= 14 40...equi done
computing for (s,i)= 14 41...equi done
computing for (s,i)= 14 42...equi done
computing for (s,i)= 14 43...equi done
computing for (s,i)= 14 44...equi done
computing for (s,i)= 14 45...equi done
computing for (s,i)= 14 46...equi done
computing for (s,i)= 14 47...equi done
computing for (s,i)= 14 48...equi done
computing for (s,i)= 14 4

computing for (s,i)= 16 44...equi done
computing for (s,i)= 16 45...equi done
computing for (s,i)= 16 46...equi done
computing for (s,i)= 16 47...equi done
computing for (s,i)= 16 48...equi done
computing for (s,i)= 16 49...equi done
computing for (s,i)= 16 50...equi done
computing for (s,i)= 16 51...equi done
computing for (s,i)= 16 52...equi done
computing for (s,i)= 16 53...equi done
computing for (s,i)= 16 54...equi done
computing for (s,i)= 16 55...equi done
computing for (s,i)= 16 56...equi done
computing for (s,i)= 16 57...equi done
computing for (s,i)= 16 58...equi done
computing for (s,i)= 16 59...equi done
computing for (s,i)= 16 60...equi done
computing for (s,i)= 16 61...equi done
computing for (s,i)= 16 62...equi done
computing for (s,i)= 16 63...equi done
computing for (s,i)= 16 64...equi done
computing for (s,i)= 16 65...equi done
computing for (s,i)= 16 66...equi done
computing for (s,i)= 16 67...equi done
computing for (s,i)= 16 68...equi done
computing for (s,i)= 16 6

computing for (s,i)= 18 65...equi done
computing for (s,i)= 18 66...equi done
computing for (s,i)= 18 67...equi done
computing for (s,i)= 18 68...equi done
computing for (s,i)= 18 69...equi done
computing for (s,i)= 18 70...equi done
computing for (s,i)= 18 71...equi done
computing for (s,i)= 18 72...equi done
computing for (s,i)= 18 73...equi done
computing for (s,i)= 18 74...equi done
computing for (s,i)= 18 75...equi done
computing for (s,i)= 18 76...equi done
computing for (s,i)= 18 77...equi done
computing for (s,i)= 18 78...equi done
computing for (s,i)= 18 79...equi done
computing for (s,i)= 18 80...equi done
computing for (s,i)= 18 81...equi done
computing for (s,i)= 18 82...computing for (s,i)= 18 83...computing for (s,i)= 18 84...computing for (s,i)= 18 85...computing for (s,i)= 18 86...computing for (s,i)= 18 87...computing for (s,i)= 18 88...computing for (s,i)= 18 89...computing for (s,i)= 18 90...computing for (s,i)= 18 91...computing for (s,i)= 18 92...computing for (s,i

computing for (s,i)= 20 80...computing for (s,i)= 20 81...computing for (s,i)= 20 82...computing for (s,i)= 20 83...computing for (s,i)= 20 84...computing for (s,i)= 20 85...computing for (s,i)= 20 86...computing for (s,i)= 20 87...computing for (s,i)= 20 88...computing for (s,i)= 20 89...computing for (s,i)= 20 90...computing for (s,i)= 20 91...computing for (s,i)= 20 92...computing for (s,i)= 20 93...computing for (s,i)= 20 94...computing for (s,i)= 20 95...computing for (s,i)= 20 96...computing for (s,i)= 20 97...computing for (s,i)= 20 98...computing for (s,i)= 20 99...computing for (s,i)= 21 1...equi done
computing for (s,i)= 21 2...equi done
computing for (s,i)= 21 3...equi done
computing for (s,i)= 21 4...equi done
computing for (s,i)= 21 5...equi done
computing for (s,i)= 21 6...equi done
computing for (s,i)= 21 7...equi done
computing for (s,i)= 21 8...equi done
computing for (s,i)= 21 9...equi done
computing for (s,i)= 21 10...equi done
computing for (s,i)= 21 11...equi done


computing for (s,i)= 23 10...equi done
computing for (s,i)= 23 11...equi done
computing for (s,i)= 23 12...equi done
computing for (s,i)= 23 13...equi done
computing for (s,i)= 23 14...equi done
computing for (s,i)= 23 15...equi done
computing for (s,i)= 23 16...equi done
computing for (s,i)= 23 17...equi done
computing for (s,i)= 23 18...equi done
computing for (s,i)= 23 19...equi done
computing for (s,i)= 23 20...equi done
computing for (s,i)= 23 21...equi done
computing for (s,i)= 23 22...equi done
computing for (s,i)= 23 23...equi done
computing for (s,i)= 23 24...equi done
computing for (s,i)= 23 25...equi done
computing for (s,i)= 23 26...equi done
computing for (s,i)= 23 27...equi done
computing for (s,i)= 23 28...equi done
computing for (s,i)= 23 29...equi done
computing for (s,i)= 23 30...equi done
computing for (s,i)= 23 31...equi done
computing for (s,i)= 23 32...equi done
computing for (s,i)= 23 33...equi done
computing for (s,i)= 23 34...equi done
computing for (s,i)= 23 3